In [ ]:
library(org.Hs.eg.db) 
library (ClusterGVis)
library(SeuratObject)
library(Seurat)
library (dplyr)
library(MAST)
library(jjAnno)

In [ ]:
output_dir <- "/path/"
seu <- readRDS("/path/SLE_Bcell.rds")
DefaultAssay(seu) 
Layers(seu[["RNA"]])  

naive_b <- subset(seu, subset = Cluster == "Naive_B")

# naive_b
Idents(naive_b) <- naive_b@meta.data$Sample
naive_b_marker <- FindAllMarkers(naive_b,
                                  assay = "RNA",
                                  test.use = 'MAST',
                                  latent.vars = "Sample",
                                  only.pos = TRUE,
                                  min.pct = 0.25,
                                  logfc.threshold = log2(1.5)) 

st.data <- prepareDataFromscRNA(object = naive_b,
                               diffData = naive_b_marker,
                               group.by = c("Sample"), 
                               showAverage = TRUE) 
df <- st.data$wide.res
head(df)

enrich <- enrichCluster(object = st.data,
                        OrgDb = org.Hs.eg.db, 
                        type = "BP", 
                        organism = "hsa", 
                        pvalueCutoff = 0.5, 
                        topn = 5,
                        seed = 5201314) 

markGenes <-atypical_b_marker %>%
  group_by(cluster) %>%
  top_n(n = 10, wt = avg_log2FC) %>%
  pull(gene) %>%
  unique()

pdf(file.path(output_dir, "NaiveB_ByTimepoint.pdf"),height = 16,width = 14,onefile = F)
visCluster(object = st.data,
           plot.type = "both",
           column_names_rot = 45,
           show_row_dend = F,
           annoTerm.data = enrich,
           line.side = "left",
           cluster.order = c(4, 5, 6, 2, 3, 1),
           sample.col = c("#3A1F75", "#9FB9EF", "#F8E66D", "#2F8BA1", "#B8EFFC", "#0D9D11"),
           go.col = rep(c("black","black","black","black","black","black"),each = 5)
          )
dev.off()